In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from_date = '2018-08-13T00:00:00'
to_date = '2018-08-14T00:00:00'

In [4]:
import pandas as pd
from datetime import datetime
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from models.article import Article

KeyError: 'DB_HOST'

In [ ]:
articles = Article.load_articles_from_db(from_date, to_date)
len(articles)

In [50]:
# Create tf_idf matrix from articles
test_url = 'http://www.dailymail.co.uk/news/article-6054347/More-260-revellers-injured-Spanish-pop-festival-pier-collapses-sea.html'
test_article = Article(test_url, '', '', '', datetime.now())


# List of named entities
named_entities_list = list(map(lambda x: ' '.join(x.named_entities), articles))
named_entities_list.append(' '.join(test_article.named_entities))

# TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(named_entities_list)

# Fit KNN
nbrs = NearestNeighbors(n_neighbors=9) 
nbrs.fit(tfidf_matrix)

# Predict
test_row = tfidf_matrix.getrow(len(named_entities_list) - 1)
distances, indices = nbrs.kneighbors(test_row)

# Format predictions
similar_articles = []
for idx in indices.flatten()[1:]:
    similar_articles.append(articles[idx])
    
df = pd.DataFrame({
    'distance': distances.flatten()[1:],
    'titles': list(map(lambda x: x.title, similar_articles)),
    'named_entities': list(map(lambda x: x.named_entities, similar_articles)),
    'url': list(map(lambda x: x.url, similar_articles)),

})
pd.set_option('display.max_colwidth', -1)
print(df)

   distance  \
0  0.852485   
1  0.941521   
2  0.981215   
3  1.043238   
4  1.089464   
5  1.127688   
6  1.188229   
7  1.298551   

                                                                                    titles  \
0  Hundreds injured at Spanish music festival as pier collapses "after crowd told to jump"   
1  Hundreds injured after platform falls into sea at Vigo music festival                     
2  Spain festival collapse in Vigo injures dozens                                            
3  Hundreds injured as Spanish festival platform collapses into sea                          
4  Boardwalk collapses in Spanish city during concert, 260 hurt                              
5  Hundreds injured as festival platform collapses into sea in Vigo, northwest Spain         
6  Platform collapses into sea at Vigo music festival in Spain – video                       
7  Northern California man shoots, kills 3 kids and himself                                  

                 